In [ ]:
using Juliana

# Config

In [ ]:
patient_ID = "water_slab_with_cavity"
preabsorber_setting = "AUTO"

output_dir = patient_ID
mkpath(output_dir)

work_dir = "tmp"
mkpath(work_dir);

In [ ]:
path_to_fiona = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
path_to_fiona_jar = "$(path_to_fiona)/ch.psi.ftpp.standalone.planner-1.0.10.jar"

tps = Juliana.FionaStandalone.FionaStandaloneTps(path_to_fiona, path_to_fiona_jar, work_dir);

In [ ]:
dose_resolution = 0.35f0 # cm
target_dose = 1.f0;

In [ ]:
gantry_angles      = [90.f0, 90.f0,  90.f0,  90.f0]
couch_angles       = [ 0.f0, 90.f0, 180.f0, 270.f0]
nozzle_extractions = [20.f0, 20.f0,  20.f0,  20.f0];

# Build phantom

In [ ]:
ct, target = Juliana.build_water_slab_with_cavity();

structures = Dict{String, Juliana.Structure}(
    "whole_body" => target,
);

In [ ]:
target.points

In [ ]:
ct, shifted_structures, _ = Juliana.shift_to_zero(ct, structures, [], calculate_distances=false);

In [ ]:
shifted_structures["whole_body"].points

# Build a treatment plan using Fiona standalone.

In [ ]:
plan = Juliana.FionaStandalone.place_spots(
    tps,
    ct,
    target,
    preabsorber_setting,
    gantry_angles,
    couch_angles,
    nozzle_extractions,
);

In [ ]:
dose = Juliana.FionaStandalone.calculate_dose(tps, dose_resolution, ct, plan);
dose = Juliana.ScalarGrid(
    Juliana.normalise_dose(dose.data, convert.(Float32, target.mask), target_dose),
    dose.grid,
);

In [ ]:
doses = Dict{String, Juliana.ScalarGrid}("A" => dose);

# Export to DICOM

In [ ]:
study_instance_UID = Juliana.get_study_instance_uid(patient_ID);

In [ ]:
Juliana.dicom_export_to_directory(
    ct,
    structures,
    output_dir,
    study_instance_UID,
    patient_ID,
    doses,
);

In [ ]:
rm(work_dir, recursive=true)